# Load the libraries

In [ ]:
import math, re, os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

# Enable the TPU

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Load data

In [ ]:
# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path()
# Configuration
NUM_CLASSES=7
EPOCHS = 10
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
IMG_SIZE = 600

In [ ]:
GCS_DS_PATH

In [ ]:
!gsutil ls $GCS_DS_PATH

In [ ]:
train_dir="../input/plant-pathology-2021-fgvc8/train_images/"
test_dir="../input/plant-pathology-2021-fgvc8/test_images/"
df_train=pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df_sub = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')


In [ ]:
df_sub.head()

In [ ]:
df_train.head()

In [ ]:
def format_train_path(fname):
    return GCS_DS_PATH+'/train_images/'+fname

In [ ]:
train_paths = df_train.image.apply(format_train_path)


Make the label encodig

In [ ]:
labels = df_train['labels'].str.split(" ").apply(pd.Series, 1).stack()
labels.index = labels.index.droplevel(-1)
target_labels = pd.get_dummies(labels).groupby(level=0).sum()
target_labels.head()

In [ ]:
train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    train_paths, target_labels, test_size=0.1, random_state=42)

In [ ]:
'''
def decode_image(filename, label=None, image_size=(IMG_SIZE, IMG_SIZE)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    print(image)
    image = tf.keras.preprocessing.image.array_to_img(image)
    image = tf.keras.preprocessing.image.random_shift(image, 0.3, 0.3),
    image = tf.keras.preprocessing.image.random_rotation(image, rg=180)
    image = tf.keras.preprocessing.image.random_brightness(image, [0.5,0.1,0.15,0.2,0.25])
    image = tf.keras.preprocessing.image.random_zoom(image,[0.5,0.1,0.15,0.2,0.25])
    image = tf.keras.preprocessing.image.random_shear(image, intensity=10)
    
    if label is None:
        return image
    else:
        return image, label
'''

In [ ]:
STEPS_PER_EPOCH = train_paths.shape[0] // BATCH_SIZE

# Image Pre-processing

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])
def decode_image(filename, label=None, image_size=(IMG_SIZE, IMG_SIZE)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None, seed=42):
    image = tf.expand_dims(image, 0)
    image = data_augmentation(image)[0]
    seed = tf.random.experimental.stateless_split([seed,IMG_SIZE], num=1)[0, :]
    image = tf.image.stateless_random_brightness(image, max_delta=0.2, seed=seed)
    #image = tf.image.stateless_random_contrast(image,0.01,0.1, seed=seed)
    
    if label is None:
        return image
    else:
        return image, label

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .cache()
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)


Create function for varying learning rate

In [ ]:
def build_lrfn(lr_start=0.00001, lr_max=0.00005,lr_min=0.0000105, lr_rampup_epochs=5,lr_sustain_epochs=0, lr_exp_decay=.5):    
    lr_max = lr_max * strategy.num_replicas_in_sync
    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs- lr_sustain_epochs) + lr_min
        return lr
    return lrfn

Metrics

In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
EarlyStopping=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=10,verbose=True, mode="min")

# Build the model

In [ ]:
from tensorflow.keras.applications import EfficientNetB7

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB7
input_shape = [IMG_SIZE, IMG_SIZE, 3]
# instantiating the model in the strategy scope creates the model on the TPU
def build_model():
    base_model = EfficientNetB7(include_top=False, input_shape=input_shape ,weights="imagenet", drop_connect_rate=0.4)
    # Freeze the pretrained weights
    for layer in base_model.layers:
            if not isinstance(layer, layers.BatchNormalization):
                layer.trainable = True
    model = Sequential()
    model.add(layers.BatchNormalization(input_shape=input_shape))
    model.add(base_model)
    # Rebuild top
    model.add(layers.GlobalAveragePooling2D(name="avg_pool"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2, name="top_dropout1"))
    model.add(layers.Dense(512,activation='relu'))
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dense(32,activation='relu'))
    model.add(layers.Dense(NUM_CLASSES, activation="sigmoid", name="pred"))
    opt = Adam(lr=0.001)
    metrics=[f1]
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=metrics )
    return model

In [ ]:
with strategy.scope():
    model = build_model()
model.summary()

# Train the model

In [ ]:
history=model.fit(train_dataset,steps_per_epoch=STEPS_PER_EPOCH,epochs=30,validation_data=valid_dataset,callbacks=[lr_schedule,EarlyStopping],verbose=1)

In [ ]:
model.save("v4.h5")

# Make predictions

In [ ]:
test_dataset = (
    tf.data.Dataset
    .list_files(GCS_DS_PATH+'/test_images/'+"*")
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

In [ ]:
pred = model.predict(test_dataset)

In [ ]:
pred

In [ ]:
pred.round()

process the predictions

In [ ]:
label_names = target_labels.columns
def get_labels_from_pred(preds):
    preds = preds.round()
    lab = []
    for row in preds:
        l = []
        for i,v in enumerate(row):
            if v == 1:
                l.append(i)
        lab.append(" ".join(label_names[l]))
    return lab

In [ ]:
pred_labels = get_labels_from_pred(pred)

In [ ]:
df_sub["labels"] = pred_labels

In [ ]:
df_sub.head()

In [ ]:
df_sub.to_csv("submission.csv", index=False, encoding='utf-8')